tissue_specific

In [106]:
import pysam as ps
import numpy as np
import collections
import matplotlib.pyplot as plt
from numpy.core.defchararray import array, center
import pysam as ps
import numpy as np
from collections import defaultdict
from scipy.ndimage.measurements import label, standard_deviation
from scipy.stats.mstats_basic import kstest, normaltest
from sklearn.cluster import KMeans
import sys
from scipy.signal import savgol_filter
import math
from subprocess import call
import os.path
from utils import Gene, TSS, Point
from scipy import stats
from sklearn import svm
import sympy
import math
from math import e
import random
from tensorflow.keras import Model
from tensorflow.keras.models import load_model
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K
from sklearn.model_selection import KFold
from tensorflow.keras.layers import Input,Conv2D,Activation,Dense,Lambda,Flatten,Embedding,PReLU,BatchNormalization,Bidirectional,LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import KFold
from tensorflow.keras import backend as K
import copy
from sklearn.utils import shuffle

In [107]:
dic = {
    'gene':'../data/gene/GRCh37.gene.bed',
    'non_gene':'../data/gene/non_gene_1234567.bed',
    'non_gene_4':'../data/gene/non_gene_4.bed',
    'non_gene_2':'../data/gene/non_gene_2.bed',
    # 'fasta':'/home/jiay/Desktop/hg19/hg19.fa',
    'bam1234567':'../data/051_1234567.bam',
    'TSS_low':'../data/gene/low_expressed.bed',
    'TSS_HK':'../data/gene/HK.bed',
    'TSS_silent':'../data/gene/silent_gene_TSS.bed',
    'ATAC_hema':'../data/gene/ATAC_hema.bed',
    'DNase_hema':'../data/gene/DNase_hema.bed',
    'ATAC_colon':'../data/gene/ATAC/colon.bed',
    'ATAC_heart':'../data/gene/ATAC/heart.bed',
    'ATAC_bcell':'../data/gene/ATAC/bcell.bed',
    'ATAC_brain':'../data/gene/ATAC/brain.bed',
    'ATAC_leukocyte':'../data/gene/ATAC/leukocyte.bed',
    'ATAC_liver':'../data/gene/ATAC/liver.bed',
    'ATAC_lung':'../data/gene/ATAC/lung.bed',
    'ATAC_skin':'../data/gene/ATAC/skin.bed',
    'ATAC_stomach':'../data/gene/ATAC/stomach.bed',
    'DNase_bcell':'../data/gene/DNase/bcell.bed',
    'DNase_brain':'../data/gene/DNase/brain.bed',
    'DNase_fibroblast_of_dermis':'../data/gene/DNase/fibroblast_of_dermis.bed',
    'DNase_hepatocyte':'../data/gene/DNase/hepatocyte.bed',
    'DNase_lung':'../data/gene/DNase/lung.bed',
    'DNase_myeloid_progenitor':'../data/gene/DNase/myeloid_progenitor.bed',
    'DNase_small_intestine':'../data/gene/DNase/small_intestine.bed',
    'DNase_tcell':'../data/gene/DNase/tcell.bed',
    'DNase_stomach':'../data/gene/DNase/stomach.bed',
    }
TSS_test = []
with open(dic['DNase_stomach'],'r') as f:
    for line in f:
        ll = line.strip().split('\t')
        if ll[0] in ['1','2','3','4','5','6','7'] and ll[-1] in ['1']:
            TSS_test.append(TSS(ll[0], int(ll[1])))
# with open(dic['ATAC_hema'],'r') as f:
#     for line in f:
#         ll = line.strip().split('\t')
#         if ll[0] in ['1']:
#             TSS_test.append(TSS(ll[0], int((int(ll[1])+int(ll[2]))/2)))
# with open(dic['non_gene_4'],'r') as f:
#     for line in f:
#         ll = line.strip().split('\t')
#         TSS_test.append(TSS(ll[0], int(ll[1]), int(ll[2])))
print(len(TSS_test))
up = down = 1000

36407


In [108]:
TSS_test = shuffle(TSS_test)[:3000]

In [109]:
test_data = []
testbam = ps.AlignmentFile(dic['bam1234567'])#lstm
test_matrix = []
for j, tss in enumerate(TSS_test):
    chrom = tss.chrom
    start = tss.pos - up
    end = tss.pos + down
    up_end = np.zeros(up+down, dtype= int)
    down_end = np.zeros(up+down, dtype= int)
    long = np.zeros(up+down, dtype= int)
    short = np.zeros(up+down, dtype= int)
    cov = np.zeros(up+down, dtype= int)
    wps = np.zeros(up+down, dtype=float)
    win = 120
    for r in testbam.fetch(chrom[-1], start-500, end + 500):
        if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse:
            if r.reference_start + abs(r.isize) < start:
                continue
            if r.reference_start >= end:
                continue
            ss = r.reference_start - start
            ee = r.reference_start - start + abs(r.isize)
            if ss >= 0:
                up_end[ss] += 1
            else:
                ss = 0
            if ee < end - start:
                down_end[ee] += 1
            else:
                ee = end - start
            for i in range(ss, ee):
                cov[i] += 1
            if 200 >= abs(r.isize) > 130:
                for i in range(ss, ee):
                    long[i] += 1
            if abs(r.isize) <= 130:
                for i in range(ss, ee):
                    short[i] += 1
            # wps_total
            region1 = int(max(0, ss + win/2))
            region2 = int(min(ee - win/2, end-start))
            i = region1
            while i < region2:
                wps[i] += 1
                i = i+1
            # wps_part
            region1 = int(max(0, ss - win/2))
            region2 = int(min(end-start, ss + win/2))
            i = region1
            while i < region2:
                wps[i] -= 1
                i = i + 1
            # wps_part
            region1 = int(max(ee - win/2, 0))
            region2 = int(min(ee + win/2, end-start))
            i = region1
            while i < region2:
                wps[i] -= 1
                i = i+1
    k = 0
    win = 40
    feature_win = np.zeros((int((up+down)/win), 4), dtype= int)
    while k < (up+down)/win:
        ss = k * win
        ee = k * win + win
        ff = []
        ff.append(int(round(np.mean(cov[ss:ee]))))
        ff.append(int(round(np.mean(long[ss:ee]-short[ss:ee]))))
        ff.append(int(round(np.sum(abs(up_end[ss:ee]-down_end[ss:ee])))))
        ff.append(int(round(np.mean(wps[ss:ee]))))
        feature_win[k] = np.array(ff)
        k = k + 1
    test_matrix.append(feature_win)
test_matrix = np.array(test_matrix)

lstm_test = []
for mat in test_matrix:
    lstm_test.append(mat)
lstm_test = np.array(lstm_test)
'''cnn_input'''
test_data = []
for j, tss in enumerate(TSS_test):
    chrom = tss.chrom
    start = tss.pos - up
    end = tss.pos + down
#    distribution_matrix = np.zeros((int(up+down), 200), dtype=int)
    distribution_matrix = np.zeros((200,200),dtype=int)
    for r in testbam.fetch(chrom[-1], start-500, end + 500):
        if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse and 50 < abs(r.isize) < 250:
            if r.reference_start + abs(r.isize) < start:
                continue
            if r.reference_start >= end:
                continue
            if r.reference_start < start:
                continue
            if r.reference_start + abs(r.isize) > end:
                continue
            ss = max(0, r.reference_start - start)
            relative_isize = abs(r.isize)-50
            distribution_matrix[ss//10,relative_isize] += 1
    test_data.append(distribution_matrix)
test_data = np.array(test_data)

cnn_test = []
for mat in test_data:
    cnn_test.append(mat)
cnn_test = np.array(cnn_test)


In [110]:
'''create model function'''
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, concatenate, Convolution1D, MaxPooling1D, Flatten, Bidirectional,Dropout, AveragePooling1D
from tensorflow.keras.models import Model
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
def create_model():
    cnn_input = Input(shape=(cnn_test.shape[1], cnn_test.shape[2]), name='cnn_input')
    lstm_input = Input(shape=(lstm_test.shape[1], lstm_test.shape[2]), name='lstm_input')
    conv1 = Convolution1D(filters=64, kernel_size=2, strides=2, activation='relu', padding='same',name='conv1')(cnn_input)
    conv1 = AveragePooling1D(pool_size=2, strides=2)(conv1)
    cnn_output = Model(inputs=cnn_input, outputs=conv1)
    main_input = concatenate([cnn_output.output, lstm_input])

    lstm_out = Bidirectional(LSTM(50, return_sequences=True),name='0')(main_input)
    conv = Convolution1D(filters=100, kernel_size=3, activation='relu',strides=1,padding='same',name='1')(lstm_out)
    pool = MaxPooling1D(pool_size=2, strides=2,name='2')(conv)
    drop = Dropout(0.2)(pool)
    flatten = Flatten()(drop)
    dense = Dense(300, activation='relu', kernel_regularizer=None, bias_regularizer=None,name='4')(flatten)
    drop = Dropout(0.2)(dense)
    main_output = Dense(1, activation='sigmoid', kernel_regularizer=None, bias_regularizer=None,name='6')(drop)

    model = Model(inputs=[cnn_output.input, lstm_input], outputs=main_output)
    return model

In [111]:
model_a = load_model('../model/model_a_hk27_150_ns.h5', compile=False)
model_b = load_model('../model/model_b_hk27_150_ns.h5', compile=False)
re_a = model_a([cnn_test,lstm_test]).numpy()
re_b = model_b([cnn_test,lstm_test]).numpy()
re = (re_a + re_b) / 2
count = 0
for rr in re:
    if rr >=0.5:
        count += 1
print("hk_ns:",count/len(re_a))
model_a = load_model('../model/model_a_hk27_150.h5', compile=False)
model_b = load_model('../model/model_b_hk27_150.h5', compile=False)
re_a = model_a([cnn_test,lstm_test]).numpy()
re_b = model_b([cnn_test,lstm_test]).numpy()
re = (re_a + re_b) / 2
count = 0
for rr in re:
    if rr >=0.5:
        count += 1
print("hk:",count/len(re_a))
model_a = load_model('../model/model_a_hema27_150_ns.h5', compile=False)
model_b = load_model('../model/model_b_hema27_150_ns.h5', compile=False)
re_a = model_a([cnn_test,lstm_test]).numpy()
re_b = model_b([cnn_test,lstm_test]).numpy()
re = (re_a + re_b) / 2
count = 0
for rr in re:
    if rr >=0.5:
        count += 1
print("hema_ns:",count/len(re_a))
model_a = load_model('../model/model_a_hema27_150.h5', compile=False)
model_b = load_model('../model/model_b_hema27_150.h5', compile=False)
re_a = model_a([cnn_test,lstm_test]).numpy()
re_b = model_b([cnn_test,lstm_test]).numpy()
re = (re_a + re_b) / 2
count = 0
for rr in re:
    if rr >=0.5:
        count += 1
print("hema:",count/len(re_a))

hk_ns: 0.2776666666666667
hk: 0.26366666666666666
hema_ns: 0.413
hema: 0.4533333333333333
